In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# Cargar .env
load_dotenv()

# Inicializar cliente con GitHub Models
client = OpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("GITHUB_TOKEN")
)

def chat():
    mensajes = [
        {"role": "system", "content": "Eres el asistente virtual de EcoMarket."}
    ]

    print("Chatbot iniciado 🤖 (escribe 'salir' para terminar)\n")

    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "salir":
            print("Chatbot finalizado 👋")
            break

        mensajes.append({"role": "user", "content": user_input})

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",   # 👈 en GitHub Models suele ser gpt-4o-mini
                messages=mensajes,
                temperature=0.3,
                max_tokens=200
            )
            reply = response.choices[0].message.content
            print("Bot:", reply, "\n")

            mensajes.append({"role": "assistant", "content": reply})
        except Exception as e:
            print("Error:", e)

if __name__ == "__main__":
    chat()


Chatbot iniciado 🤖 (escribe 'salir' para terminar)

Bot: ¡Hola! ¿Cómo puedo ayudarte hoy en EcoMarket? 

Chatbot finalizado 👋
